In [1]:
import os
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='grayscale', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

def resnet_block(x, filters):
    shortcut = x

    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), padding='same')(shortcut)
    
    x = layers.add([x, shortcut])
    x = layers.Activation(mish)(x)
    return x

def create_resnet_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=(7, 7), padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.MaxPooling2D(pool_size=(3, 3))(x)
    

    filter_sizes = [64, 128, 256, 512]
    for filters in filter_sizes:
        for _ in range(2):
            x = resnet_block(x, filters)
        x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    output_layer = layers.Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs=input_layer, outputs=output_layer)

folder_path = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\Pupil\crop_img"
results = pd.read_excel("results.xlsx", header=None)
files = glob.glob(os.path.join(folder_path, '*.jpg'))

right_images = [file for file in files if 'right' in os.path.basename(file).lower()]
left_images = [file for file in files if 'left' in os.path.basename(file).lower()]

right_images = np.array([load_and_preprocess_image(img_path) for img_path in right_images])
left_images = np.array([load_and_preprocess_image(img_path) for img_path in left_images])
labels = results[0].values
num_classes = len(np.unique(labels))
labels = to_categorical(labels, num_classes=num_classes)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_train_accuracies = []
fold_val_accuracies = []

model_save_folder = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\0930_resnet_models"
if not os.path.exists(model_save_folder):
    os.makedirs(model_save_folder)

csv_save_folder = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\0930_resnet_predictions"
if not os.path.exists(csv_save_folder):
    os.makedirs(csv_save_folder)

for fold_idx, (train_index, val_index) in enumerate(kf.split(right_images)):
    right_images_train, right_images_val = right_images[train_index], right_images[val_index]
    left_images_train, left_images_val = left_images[train_index], left_images[val_index]
    labels_train, labels_val = labels[train_index], labels[val_index]

    right_eye_model = create_resnet_model((128, 128, 1))
    left_eye_model = create_resnet_model((128, 128, 1))

    combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output])

    x = layers.Dense(256, activation=mish)(combined_input)
    x = layers.Dropout(0.5)(x)
    output_layer = layers.Dense(num_classes, activation='softmax')(x)

    combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input], outputs=output_layer)

    combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = combined_model.fit([right_images_train, left_images_train], labels_train, 
                                 epochs=30, batch_size=1, validation_data=([right_images_val, left_images_val], labels_val))

    train_accuracy = history.history['accuracy'][-1]
    val_accuracy = history.history['val_accuracy'][-1]
    fold_train_accuracies.append(train_accuracy)
    fold_val_accuracies.append(val_accuracy)

    print(f"Fold {fold_idx+1} - Train Accuracy: {train_accuracy}, Validation Accuracy: {val_accuracy}")

    combined_model.save(os.path.join(model_save_folder, f"fold_{fold_idx+1}_combined_model.h5"))

    predictions = combined_model.predict([right_images_val, left_images_val])
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(labels_val, axis=1)

    data = {
        'Right Image': [os.path.basename(files[idx]) for idx in val_index],
        'Left Image': [os.path.basename(files[idx]) for idx in val_index],
        'Actual Class': actual_classes,
        'Predicted Class': predicted_classes
    }
    df = pd.DataFrame(data)
    df.to_csv(os.path.join(csv_save_folder, f"fold_{fold_idx+1}_predictions.csv"), index=False)

avg_train_accuracy = np.mean(fold_train_accuracies)
avg_val_accuracy = np.mean(fold_val_accuracies)

print(f'Average Train Accuracy: {avg_train_accuracy}')
print(f'Average Validation Accuracy: {avg_val_accuracy}')

Epoch 1/30
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 441s 155ms/step - accuracy: 0.7678 - loss: 1.2115 - val_accuracy: 0.8700 - val_loss: 0.3818
Epoch 2/30
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 440s 157ms/step - accuracy: 0.9405 - loss: 0.2058 - val_accuracy: 0.9800 - val_loss: 0.0565
Epoch 3/30
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 441s 158ms/step - accuracy: 0.9864 - loss: 0.0584 - val_accuracy: 0.9900 - val_loss: 0.0258
Epoch 4/30
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 434s 155ms/step - accuracy: 0.9825 - loss: 0.0862 - val_accuracy: 0.9829 - val_loss: 0.0451
Epoch 5/30
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 427s 153ms/step - accuracy: 0.9869 - loss: 0.0464 - val_accuracy: 0.9857 - val_loss: 0.0436
Epoch 6/30
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 418s 149ms/step - accuracy: 0.9914 - loss: 0.0300 - val_accuracy: 0.7600 - val_loss: 1.3844
Epoch 7/30
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 438s 156ms/step - accuracy: 0.9903 - loss: 0.0295 - val_accuracy: 0.9886 - val_loss: 0.0318
Epoch 8/30
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 429s 153ms/step - ac

KeyboardInterrupt: 

# 20에폭, RESNET

In [4]:
import os
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

# 이미지 로드 및 전처리 함수
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='grayscale', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# ResNet 블록 (Residual Block)
def resnet_block(x, filters, downsample=False):
    shortcut = x
    strides = (2, 2) if downsample else (1, 1)

    # 첫 번째 Conv 레이어
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)

    # 두 번째 Conv 레이어
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)

    # shortcut 경로를 맞추기 위한 Conv 레이어
    if downsample or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
    
    x = layers.add([x, shortcut])
    x = layers.Activation(mish)(x)
    return x

# ResNet-18 모델 생성
def create_resnet18_model(input_shape):
    input_layer = Input(shape=input_shape)

    # 초기 Conv 레이어
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # Residual 블록들
    filter_sizes = [64, 128, 256, 512]
    num_blocks = [2, 2, 2, 2]  # ResNet-18은 각 필터 크기에서 2개의 블록이 존재
    for filters, blocks in zip(filter_sizes, num_blocks):
        for i in range(blocks):
            x = resnet_block(x, filters, downsample=(i == 0 and filters != 64))  # 첫 블록만 다운샘플링
    
    x = layers.GlobalAveragePooling2D()(x)  # 최종적으로 Global Average Pooling
    return Model(inputs=input_layer, outputs=x)

# 파일 경로 설정 및 데이터 로드
folder_path = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\Pupil\crop_img"
results = pd.read_excel("results.xlsx", header=None)
files = glob.glob(os.path.join(folder_path, '*.jpg'))

right_images = [file for file in files if 'right' in os.path.basename(file).lower()]
left_images = [file for file in files if 'left' in os.path.basename(file).lower()]

right_images = np.array([load_and_preprocess_image(img_path) for img_path in right_images])
left_images = np.array([load_and_preprocess_image(img_path) for img_path in left_images])
labels = results[0].values
num_classes = len(np.unique(labels))
labels = to_categorical(labels, num_classes=num_classes)

# 교차 검증 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_train_accuracies = []
fold_val_accuracies = []

# 모델 저장 폴더 생성
model_save_folder = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\0930_resnet18_models"
if not os.path.exists(model_save_folder):
    os.makedirs(model_save_folder)

# 예측 결과 저장 폴더 생성
csv_save_folder = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\0930_resnet18_predictions"
if not os.path.exists(csv_save_folder):
    os.makedirs(csv_save_folder)

# 각 폴드별로 학습
for fold_idx, (train_index, val_index) in enumerate(kf.split(right_images)):
    right_images_train, right_images_val = right_images[train_index], right_images[val_index]
    left_images_train, left_images_val = left_images[train_index], left_images[val_index]
    labels_train, labels_val = labels[train_index], labels[val_index]

    # 오른쪽과 왼쪽 눈에 각각 ResNet-18 모델 적용
    right_eye_model = create_resnet18_model((128, 128, 1))
    left_eye_model = create_resnet18_model((128, 128, 1))

    # 두 눈에서 추출한 특성 결합
    combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output])

    # 완전 연결 레이어
    x = layers.Dense(256, activation=mish)(combined_input)
    x = layers.Dropout(0.5)(x)
    output_layer = layers.Dense(num_classes, activation='softmax')(x)

    # 최종 모델 정의
    combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input], outputs=output_layer)

    # 모델 컴파일
    combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # 모델 학습
    history = combined_model.fit([right_images_train, left_images_train], labels_train, 
                                 epochs=20, batch_size=1, validation_data=([right_images_val, left_images_val], labels_val))

    # 폴드별 정확도 기록
    train_accuracy = history.history['accuracy'][-1]
    val_accuracy = history.history['val_accuracy'][-1]
    fold_train_accuracies.append(train_accuracy)
    fold_val_accuracies.append(val_accuracy)

    print(f"Fold {fold_idx+1} - Train Accuracy: {train_accuracy}, Validation Accuracy: {val_accuracy}")

    # 모델 저장
    combined_model.save(os.path.join(model_save_folder, f"fold_{fold_idx+1}_combined_model.h5"))

    # 예측 및 결과 저장
    predictions = combined_model.predict([right_images_val, left_images_val])
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(labels_val, axis=1)

    data = {
        'Right Image': [os.path.basename(files[idx]) for idx in val_index],
        'Left Image': [os.path.basename(files[idx]) for idx in val_index],
        'Actual Class': actual_classes,
        'Predicted Class': predicted_classes
    }
    df = pd.DataFrame(data)
    df.to_csv(os.path.join(csv_save_folder, f"fold_{fold_idx+1}_predictions.csv"), index=False)

# 전체 평균 정확도 계산
avg_train_accuracy = np.mean(fold_train_accuracies)
avg_val_accuracy = np.mean(fold_val_accuracies)

print(f'Average Train Accuracy: {avg_train_accuracy}')
print(f'Average Validation Accuracy: {avg_val_accuracy}')


Epoch 1/20
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 677s 236ms/step - accuracy: 0.6962 - loss: 0.9760 - val_accuracy: 0.8057 - val_loss: 0.6668
Epoch 2/20
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 668s 239ms/step - accuracy: 0.9437 - loss: 0.1704 - val_accuracy: 0.9014 - val_loss: 0.4072
Epoch 3/20
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 654s 234ms/step - accuracy: 0.9686 - loss: 0.1180 - val_accuracy: 0.8829 - val_loss: 0.8537
Epoch 4/20
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 645s 231ms/step - accuracy: 0.9745 - loss: 0.1363 - val_accuracy: 0.8614 - val_loss: 1.8013
Epoch 5/20
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 647s 231ms/step - accuracy: 0.9779 - loss: 0.1014 - val_accuracy: 0.8086 - val_loss: 2.6446
Epoch 6/20
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 647s 231ms/step - accuracy: 0.9722 - loss: 0.1363 - val_accuracy: 0.9186 - val_loss: 1.1451
Epoch 7/20
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 650s 232ms/step - accuracy: 0.9916 - loss: 0.0414 - val_accuracy: 0.8200 - val_loss: 2.5410
Epoch 8/20
2796/2796 ━━━━━━━━━━━━━━━━━━━━ 657s 235ms/step - ac

Fold 1 - Train Accuracy: 0.9960657954216003, Validation Accuracy: 0.9757142663002014
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 254ms/step
Epoch 1/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 677s 236ms/step - accuracy: 0.6759 - loss: 1.0553 - val_accuracy: 0.9657 - val_loss: 0.1934
Epoch 2/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 662s 237ms/step - accuracy: 0.9483 - loss: 0.1821 - val_accuracy: 0.9442 - val_loss: 0.2390
Epoch 3/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 661s 236ms/step - accuracy: 0.9737 - loss: 0.0828 - val_accuracy: 0.9142 - val_loss: 0.3589
Epoch 4/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 662s 237ms/step - accuracy: 0.9792 - loss: 0.0826 - val_accuracy: 0.8612 - val_loss: 0.6351
Epoch 5/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 665s 238ms/step - accuracy: 0.9777 - loss: 0.0957 - val_accuracy: 0.8755 - val_loss: 0.8562
Epoch 6/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 662s 237ms/step - accuracy: 0.9923 - loss: 0.0303 - val_accuracy: 0.9499 - val_loss: 0.6573
Epoch 7/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 661s 236ms/step - accuracy: 

Fold 2 - Train Accuracy: 0.994637131690979, Validation Accuracy: 0.8912732601165771
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 254ms/step
Epoch 1/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 691s 241ms/step - accuracy: 0.6808 - loss: 1.0241 - val_accuracy: 0.9671 - val_loss: 0.1156
Epoch 2/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 673s 241ms/step - accuracy: 0.9534 - loss: 0.1690 - val_accuracy: 0.8212 - val_loss: 0.6215
Epoch 3/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 671s 240ms/step - accuracy: 0.9755 - loss: 0.0935 - val_accuracy: 0.6309 - val_loss: 3.5666
Epoch 4/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 673s 241ms/step - accuracy: 0.9696 - loss: 0.1212 - val_accuracy: 0.9313 - val_loss: 0.6975
Epoch 5/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 672s 240ms/step - accuracy: 0.9754 - loss: 0.1002 - val_accuracy: 0.8813 - val_loss: 1.1605
Epoch 6/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 673s 241ms/step - accuracy: 0.9845 - loss: 0.0653 - val_accuracy: 0.8884 - val_loss: 1.2517
Epoch 7/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 674s 241ms/step - accuracy: 0

Fold 3 - Train Accuracy: 0.994637131690979, Validation Accuracy: 0.8068669438362122
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 259ms/step
Epoch 1/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 696s 243ms/step - accuracy: 0.6969 - loss: 1.0053 - val_accuracy: 0.7668 - val_loss: 0.9088
Epoch 2/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 681s 243ms/step - accuracy: 0.9551 - loss: 0.1703 - val_accuracy: 0.8441 - val_loss: 1.0730
Epoch 3/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 679s 243ms/step - accuracy: 0.9669 - loss: 0.1435 - val_accuracy: 0.9657 - val_loss: 0.2215
Epoch 4/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 684s 245ms/step - accuracy: 0.9795 - loss: 0.0921 - val_accuracy: 0.9814 - val_loss: 0.0855
Epoch 5/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 679s 243ms/step - accuracy: 0.9763 - loss: 0.0814 - val_accuracy: 0.9142 - val_loss: 0.5089
Epoch 6/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 679s 243ms/step - accuracy: 0.9907 - loss: 0.0432 - val_accuracy: 0.9714 - val_loss: 0.2525
Epoch 7/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 679s 243ms/step - accuracy: 0

Fold 4 - Train Accuracy: 0.9917768836021423, Validation Accuracy: 0.9728183150291443
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 261ms/step
Epoch 1/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 701s 244ms/step - accuracy: 0.7048 - loss: 1.0242 - val_accuracy: 0.8155 - val_loss: 0.9589
Epoch 2/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 685s 245ms/step - accuracy: 0.9564 - loss: 0.1618 - val_accuracy: 0.8956 - val_loss: 0.7067
Epoch 3/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 685s 245ms/step - accuracy: 0.9741 - loss: 0.0837 - val_accuracy: 0.8784 - val_loss: 0.9600
Epoch 4/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 685s 245ms/step - accuracy: 0.9843 - loss: 0.0477 - val_accuracy: 0.7067 - val_loss: 4.5299
Epoch 5/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 687s 246ms/step - accuracy: 0.9770 - loss: 0.0817 - val_accuracy: 0.7439 - val_loss: 6.4428
Epoch 6/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 686s 245ms/step - accuracy: 0.9844 - loss: 0.0788 - val_accuracy: 0.7368 - val_loss: 11.4279
Epoch 7/20
2797/2797 ━━━━━━━━━━━━━━━━━━━━ 686s 245ms/step - accuracy:

Fold 5 - Train Accuracy: 0.9928494691848755, Validation Accuracy: 0.9427753686904907
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 263ms/step
Average Train Accuracy: 0.9939932823181152
Average Validation Accuracy: 0.9178896307945251
